In [ ]:
from transformers import AutoModel, AutoTokenizer, TextStreamer
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048 # Maximum number of tokens in a sequence
dtype = None # Data type of the model input tensor

model_name = "jackma-00/lora_model" # Model name

# Load the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name, 
    max_seq_length=max_seq_length, 
    dtype=dtype,
    load_in_4bit=True
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "system", "content": "You are a friendly Chatbot."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True, # Must add for generation
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)